## Unsteady Stokes Problem on a Time-Dependent Domain

In this notebook, we solve the **unsteady (instationary) Stokes problem** on a time-dependent domain $\Omega(t)$ using an unfitted finite element method.

The continuous problem is formulated as follows:

Find velocity $u(t, x)$ and pressure $p(t, x)$ such that

$$
\partial_t u - \Delta u + \nabla p = f \quad \text{in } \Omega(t),
$$

$$
\operatorname{div} u = 0 \quad \text{in } \Omega(t),
$$

with appropriate initial and boundary conditions:

- $u(0, x) = u_0(x)$ in $\Omega(0)$,
- $u = g$ on $\partial \Omega(t)$ for all $t$.

We aim to discretize this system in space using unfitted finite elements with ghost penalty stabilization, and in time using a standard time-stepping scheme (e.g., backward Euler or BDF). This formulation allows us to handle complex or moving geometries without remeshing.



In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *
import numpy as np

importing ngsxfem-2.1.2405


## Background Mesh and Domain Embedding

As in the stationary case, we begin by creating a **fixed background mesh** $\widehat{\Omega}$ such that the **physical domain** $\Omega(t)$ is always contained within it for all relevant times $t$:

$$
\Omega(t) \subset \widehat{\Omega} \quad \text{for all } t.
$$

This allows us to work on a static computational mesh while the actual domain $\Omega(t)$ may evolve in time. The geometry of $\Omega(t)$ will be described implicitly by a **level set function** $\phi(t, x)$, and the unfitted method will handle the interface between the physical and fictitious domains.


In [2]:
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=0.05)
mesh = Mesh(ngmesh)

## Time-Dependent Level Set Function

To model the motion of the domain $\Omega(t)$ over time, we now define a **time-dependent level set function** $\phi(t, x, y)$. This function implicitly describes the geometry of $\Omega(t)$ as the subdomain where $\phi(t, x, y) < 0$.

In our case, we simulate the motion of a **circle moving from left to right** with constant velocity $v$. The level set function has the form:

$$
\phi(t, x, y) = (x - v t)^2 + y^2 - r^2,
$$

which represents a circle of radius $r$ centered at $(x = v t, y = 0)$. As time $t$ increases, the circle moves along the $x$-axis with velocity $v$.

By treating $t$ as a symbolic or parameterized variable in our implementation, we ensure that $\phi(t, x, y)$ is **automatically updated** whenever the time step advances. This provides a flexible and efficient way to model moving domains without changing the mesh.


In [3]:
from ngsolve import Parameter
from time import sleep
t1 = -10
tend = 10
dt = 0.2

t = Parameter(0)
r = 0.5
vel = 0.1
levelset = CF(sqrt((x-vel*t)**2 + y**2))-r
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)

mask = IfPos(-levelset, 1.0, 0.0)
scene = Draw(mask, mesh, "levelset_mask")
scene1 = DrawDC(levelset,CF(1),CF(0), mesh, "levelset")

while(t1 <tend):
    t.Set(t1)
    scene.Redraw()
    scene1.Redraw()
    sleep(0.005)
    t1 += dt


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

### ⏱️ Semi-Discrete Formulation of the Unsteady Stokes Problem

We now formulate the **semi-discrete form** of the unsteady Stokes equations, using finite elements in space and keeping time continuous at first. For time discretization, we apply a **backward Euler scheme** with time step $\Delta t$.

Let $u^n_h$, $p^n_h$ be the discrete velocity and pressure at time step $n$, and $u^{n-1}_h$ the solution from the previous time step. We extend $u^{n-1}_h$ to the current domain $\Omega^n$ via a suitable **extension operator** (e.g., constant extension in normal direction). The semi-discrete variational formulation then reads:

> Find $(u^n, p^n) \in V^n \times Q^n := H^1(\Omega^n) \times L^2_0(\Omega^n)$
, $p^n_h \in Q_h$ such that for all $v_h \in V_h$, $q_h \in Q_h$:
>
> $$
> \frac{1}{\Delta t} (u^n_h, v_h) + a_h(u^n_h, v_h) + b_h(p^n_h, v_h) + b_h(q_h, u^n_h) = \langle f^n, v_h \rangle + \frac{1}{\Delta t} ( \tilde{u}^{n-1}_h, v_h )
> $$

Here:

- $a^n(u, v) = \nu \int_{\Omega^n} \nabla u : \nabla v \, dx$ is the **viscous bilinear form**, where $\nu$ denotes the viscosity. 
- $b^n(q, v) = - \int_{\Omega^n} q \, \nabla \cdot v \, dx$ encodes the **divergence-pressure coupling**.
- $\tilde{u}^{n-1}_h = E(u^{n-1}_h)$ is the **extension** of the previous solution $u^{n-1}_h \in H^1_0(\Omega^{n-1})$ into the current domain $\Omega^n$.
- The **extension operator** $E : H^1(\Omega^{n-1}) \rightarrow H^1(\mathcal{O}_\delta(\Omega^{n-1}))$ maps the previous solution to a neighborhood $\mathcal{O}_\delta(\Omega^{n-1})$ that contains the new domain $\Omega^n \subset \mathcal{O}_\delta(\Omega^{n-1})$. This is necessary to evaluate $u^{n-1}$ on the new domain.
- $\langle f^n, v_h \rangle = \int_{\Omega^n} f^n \cdot v_h \, dx$ is the **force term** at time $t^n$.

All integrals are taken over the current domain $\Omega^n$.

This formulation accounts for time evolution in a robust way under the unfitted finite element framework, by incorporating both an appropriate **extension of the previous solution** and **stabilization techniques** in space.


### 🔁 Construction of the Extended Domain $\mathcal{O}_\delta(\Omega^n_h)$

To ensure stability of the unfitted discretization and to properly apply the initial condition from the previous time step, we solve the problem not only on the physical domain $\Omega^n_h$, but on a slightly **extended domain** $\mathcal{O}_\delta(\Omega^n_h)$ around it.

To do so, we define two auxiliary level set functions:

- $\phi^+_{\text{extend}}$: A **level set function** whose zero contour defines the boundary of the extended domain *from the outside*.
- $\phi^-_{\text{extend}}$: A **level set function** that defines an optional inner boundary for excluding small regions from the extension zone (often unused or set to $-∞$).

The extended domain is then defined as:
$$
\mathcal{O}_\delta(\Omega^n_h) := \{ x \in \hat{\Omega} \mid \phi^+_{\text{extend}}(x) < 0 \text{ and } \phi^-_{\text{extend}}(x) > 0 \}
$$
where $\hat{\Omega}$ is the fixed background mesh domain.

---

#### 📌 Ghost Penalty Region

The **ghost penalty stabilization** is not applied on the entire background mesh, but is restricted to a **narrow ring** of elements around the physical domain:

$$
\mathcal{R}^n_{h,\delta} := \left\{ K \in \mathcal{T}_h \;\middle|\; \exists x \in K \text{ such that } \mathrm{dist}(x, \Omega^n_h) \leq \delta_h \right\} \subset \mathcal{T}_h
$$

This *ghost ring* $\mathcal{R}^n_{h,\delta}$ contains all elements that are within a small distance $\delta_h$ of the physical domain. The ghost penalty terms are only applied on the **facets of elements in this ring**, which stabilizes the discretization near the boundary without affecting the interior.

---

This setup enables:
- consistent evaluation of previous time step data $u^{n-1}$,
- robust enforcement of boundary conditions,
- stable solution of the time-dependent Stokes problem on **evolving domains**.

---

At each time step, we therefore extend the (discrete) physical domain $\Omega^n_h$ by a strip of width
$$
\delta_h = c_\delta \, \|w^n\|_\infty \, \Delta t,
$$
where $c_\delta$ is a user-defined constant, $w^n$ is the velocity at time step $n$, and $\Delta t$ is the time step size.


In [4]:
from numpy import max
import ngsolve.webgui as wg
w = max(vel)
delta = delta = w*dt*10
levelset_extended_plus = CF(sqrt((x-vel*t)**2 + y**2))-(r+delta)
levelset_extended_minus = CF(sqrt((x-vel*t)**2 + y**2))-(r-delta)
lsetp1_extended_plus = GridFunction(H1(mesh))
InterpolateToP1(levelset_extended_plus, lsetp1_extended_plus)
lsetp1_extended_minus = GridFunction(H1(mesh))
InterpolateToP1(levelset_extended_minus, lsetp1_extended_minus)


for t1 in range(0, 5):
    delta = w*dt*3
    t.Set(t1)


    lsetp1 = GridFunction(H1(mesh))
    InterpolateToP1(levelset,lsetp1)
    lsetp1_extended_plus = GridFunction(H1(mesh))
    InterpolateToP1(levelset_extended_plus, lsetp1_extended_plus)
    lsetp1_extended_minus = GridFunction(H1(mesh))
    InterpolateToP1(levelset_extended_minus, lsetp1_extended_minus)

    ci = CutInfo(mesh, lsetp1)
    hasneg = ci.GetElementsOfType(HASNEG)
    neg = ci.GetElementsOfType(NEG)
    hasif = ci.GetElementsOfType(IF)
    haspos = ci.GetElementsOfType(HASPOS)

    ci1 = CutInfo(mesh, lsetp1_extended_plus)
    hasneg1 = ci1.GetElementsOfType(HASNEG)
    neg1 = ci1.GetElementsOfType(NEG)
    hasif1 = ci1.GetElementsOfType(IF)
    haspos1 = ci1.GetElementsOfType(HASPOS)

    ci2 = CutInfo(mesh, lsetp1_extended_minus)
    hasneg2 = ci2.GetElementsOfType(HASNEG)
    neg2 = ci2.GetElementsOfType(NEG)
    hasif2 = ci2.GetElementsOfType(IF)
    haspos2 = ci2.GetElementsOfType(HASPOS)

    ba_diff = BitArray(len(hasneg1))
    ba_diff[:] = hasneg1
    ba_diff &= ~neg2

    wg.Draw(BitArrayCF(ba_diff),mesh,"elements_extended_"+str(dt))
    


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

## ✅ Constructing an Analytical Example

Now that we have defined the **extended domain** $\mathcal{O}_\delta(\Omega^n_h)$ and established a method for identifying the **facet patches** required for ghost penalty stabilization at each time step, we are ready to **solve the unsteady Stokes problem** numerically.

To verify the accuracy and stability of our method, we construct an **analytical solution** $(u_{\text{exact}}, p_{\text{exact}})$ and derive the corresponding forcing term $f$ from the equations:

$$
\frac{\partial u}{\partial t} - \nu \Delta u + \nabla p = f \quad \text{in } \Omega(t),
$$
$$
\nabla \cdot u = 0 \quad \text{in } \Omega(t).
$$

We will prescribe $u_{\text{exact}}$ and $p_{\text{exact}}$ in such a way that:
- the divergence-free condition is satisfied exactly,
- the time-dependence is smooth,
- and the domain evolution (encoded in the level set) matches the motion of the flow.

This allows us to compute the **exact right-hand side $f$** and use it to assess the **error and convergence** of the numerical solution over time.


In [5]:
c = CF((x-t)**2 + y**2)
uexact = CF((-2*y*cos(c),2*(x-t)*cos(c)))

laplace_uexact = CF((16*y*sin(c)+8*y*c*cos(c), -16*(x-t)*sin(c)-8*(x-t)*c*cos(c)))
ut_exact = CF((4*y*(t-x)*sin(c), -2*cos(c)-4*sin(c)*(x-t)**2))

laplace_uexact = CF((uexact[0].Diff(x).Diff(x) + uexact[0].Diff(y).Diff(y),uexact[1].Diff(x).Diff(x) + uexact[1].Diff(y).Diff(y)))
ut_exact = CF((uexact[0].Diff(t),uexact[1].Diff(t)))

pressure = 0
f = -laplace_uexact + ut_exact


In [6]:
order = 2
V = VectorH1(mesh, order=order,dgjumps=True)
Q = H1(mesh, order=order-1)
Z = NumberSpace(mesh)

X = V*Q*Z
(u,p,z),(v,q,z1) = X.TnT()
gfu = GridFunction(X)

In [ ]:
nu =1 
gamma_stab =100
tstart =0
tend = 1
t1 = tstart
dt = 0.001

beta0 = 10*order**2
beta2 = 10*order**2

def jump(f):
    return f - f.Other()
def jumpn(f):
    n = Normalize(grad(lsetp1))
    return Grad(f)*n - Grad(f).Other()*n

while t1 < tend:
    t.Set(t1)
    print ("\rt=", t, end="")
    
    lsetp1 = GridFunction(H1(mesh))
    InterpolateToP1(levelset,lsetp1)

    lsetp1_extended_plus = GridFunction(H1(mesh))
    InterpolateToP1(levelset_extended_plus, lsetp1_extended_plus)

    lsetp1_extended_minus = GridFunction(H1(mesh))
    InterpolateToP1(levelset_extended_minus, lsetp1_extended_minus)


    h = specialcf.mesh_size
    n = Normalize(grad(lsetp1))

    ci = CutInfo(mesh, lsetp1)
    hasneg = ci.GetElementsOfType(HASNEG)
    neg = ci.GetElementsOfType(NEG)
    hasif = ci.GetElementsOfType(IF)
    haspos = ci.GetElementsOfType(HASPOS)

    ci1 = CutInfo(mesh, lsetp1_extended_plus)
    hasneg1 = ci1.GetElementsOfType(HASNEG)
    neg1 = ci1.GetElementsOfType(NEG)
    hasif1 = ci1.GetElementsOfType(IF)
    haspos1 = ci1.GetElementsOfType(HASPOS)

    ci2 = CutInfo(mesh, lsetp1_extended_minus)
    hasneg2 = ci2.GetElementsOfType(HASNEG)
    neg2 = ci2.GetElementsOfType(NEG)
    hasif2 = ci2.GetElementsOfType(IF)
    haspos2 = ci2.GetElementsOfType(HASPOS)

    ba_diff = BitArray(len(hasneg1))
    ba_diff[:] = hasneg1
    ba_diff &= ~neg2


    dx = dCut(lsetp1, NEG, definedonelements=hasneg)
    ds = dCut(lsetp1, IF, definedonelements=hasif)
    dw_interface = dFacetPatch(definedonelements=ba_diff)

    a = BilinearForm(X)
    stokes = nu * InnerProduct(Grad(u), Grad(v))*dx - div(u)*q*dx - div(v)*p*dx
    stokes += -nu*(Grad(u)*n * v + Grad(v)*n * u) * ds + nu*gamma_stab / h * u * v* ds #nitzshe stabilization
    stokes += (q*n * u + p*n * v) * ds
    stokes += p*z1 *dx + q *z*dx
    stokes += nu*beta2*h**-2* InnerProduct(jump(u), jump(v)) * dw_interface #velocity ghost penalty
    stokes += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface #pressure ghost penalty
    a += stokes
    a += 1/dt * InnerProduct(u, v) * dx # time derivative
    a.Assemble()


    if t1 == 0:
        gfu.components[0].Set(uexact)   
    #else:
        #active_dofs_inner=GetDofsOfElements(X,hasneg)
        #active_dofs_inner &= ~active_dofs
        #print("sss:", sum(active_dofs_inner))

    ud = uexact

    active_dofs=GetDofsOfElements(X,hasneg1)
    inv = a.mat.Inverse(active_dofs)

    
    l2error = sqrt(Integrate( (gfu.components[0] - uexact) ** 2*dx, mesh ))
    print("t=", t1, "l2error :", l2error)
    #DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
    #DrawDC(lsetp1,uexact, CF((0,0)), mesh, "uexact")

    res = LinearForm(X)
    res += f * v * dx
    res += 1/dt*InnerProduct(gfu.components[0], v)*dx
    res += ud * n * q *ds -  nu*Grad(v) * n *ud * ds + nu*gamma_stab/h * ud *v*ds

    t.Set(t1+dt)
    
    res.Assemble()
    gfu.vec.data =  inv * res.vec
    #print("gfu.vec.data:", gfu.vec.data)
    t1 = t1 + dt
    
    #print("t=", t, "cnt=", cnt)
    
    #if cnt % 50 == 0:
    #sceneu.Redraw()
    #scenep.Redraw()
    #gfut.AddMultiDimComponent(gfu.vec)

t= ParameterCF, val = 0


used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


t= 0 l2error : 2.2315916844165355e-06
t= ParameterCF, val = 0.001


used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


t= 0.001 l2error : 1.9423371074876319
t= ParameterCF, val = 0.002


...

t= 0.47200000000000036 l2error : 6988.799763193621
t= ParameterCF, val = 0.473


used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


t= 0.47300000000000036 l2error : 6600.022107920462
t= ParameterCF, val = 0.474


used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


t= 0.47400000000000037 l2error : 6232.6224173865
t= ParameterCF, val = 0.475
